In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
import os
import datetime
from tqdm import tqdm
from PIL import Image

import torch
import torch.nn as nn

from torch import optim
from torch.utils.data import Dataset, DataLoader
import sys 
sys.path.append("..") 
from model.dataset import *

from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import mean_squared_error as mse
from skimage.metrics import normalized_root_mse as nrmse
from skimage.filters import threshold_otsu, threshold_multiotsu
from sklearn.metrics import roc_auc_score
from scipy.stats import wilcoxon

使用8bit图像 0-255
计算AUC
最后还是用老照片，没有用随机数
MSE计算前先除以255

In [12]:
# otsu法，分为四类，取亮度最大的一类作为阈值
def auc_img(fake_img, true_img, classes = 4): 
    # 0-255的256@256图片
    thresholds = threshold_multiotsu(true_img, classes=classes)
    binary = np.array(true_img >= thresholds[-1])
    return roc_auc_score(binary.flatten(), fake_img.flatten()/255)

def cal_metric(fake_img, target_img):
    #    fake_img, target_img = renorm(fake_img), renorm(target_img)
       AUC = auc_img(target_img, fake_img)
       MSE = mse(target_img/255, fake_img/255) #MSE计算前先除以255
       SSIM = ssim(target_img, fake_img) #, data_range=1 
       PSNR = psnr(target_img, fake_img) #, data_range=1
    #    NRMSE = nrmse(target_img, fake_img) 
       return AUC, MSE, SSIM, PSNR

In [13]:
def get_gloabl_metrics():
    truth_dir = "./DiffusionSpinalMRISynthesis/results_output_pictures/Ground_True_imgs_test_spinal/"
    pix2pix_dir = "./DiffusionSpinalMRISynthesis/results_output_pictures/1228_pix2pix_500epoch_test_spinal/"
    diff_dir = "./DiffusionSpinalMRISynthesis/results_output_pictures/1130_diffusion_500epoch_test_spinal" #老的结果
    # diff_dir = "./DiffusionSpinalMRISynthesis/results_output_pictures/1229_diffusion_randomseed666_500epoch_test_spinal_16ddim/"

    patients_id = os.listdir(diff_dir)
    df_diff_metrics = pd.DataFrame({'AUC':[], 'MSE':[], 'SSIM':[], 'PSNR':[]})
    df_pix2pix_metrics = pd.DataFrame({'AUC':[], 'MSE':[], 'SSIM':[], 'PSNR':[]})
    truth_imgs, diff_imgs, pix2pix_imgs =  [], [], []
    for patient in tqdm(patients_id):
        # print('patient:', patient)
        truth_patient_dir = os.path.join(truth_dir, patient)
        truth_imgs.extend([np.array(Image.open(os.path.join(truth_patient_dir, file))) for file in os.listdir(truth_patient_dir)])

        diff_patient_dir = os.path.join(diff_dir, patient)
        diff_imgs.extend([np.array(Image.open(os.path.join(diff_patient_dir, file)))[:,256:] for file in os.listdir(diff_patient_dir)])

        pix2pix_patient_dir = os.path.join(pix2pix_dir, patient)
        pix2pix_imgs.extend([np.array(Image.open(os.path.join(pix2pix_patient_dir, file))) for file in os.listdir(pix2pix_patient_dir)])
    # return truth_imgs, diff_imgs, pix2pix_imgs 
    for i in tqdm(range(len(truth_imgs))):
        df_diff_metrics.loc[df_diff_metrics.shape[0], :] = [*cal_metric(diff_imgs[i], truth_imgs[i])]
        df_pix2pix_metrics.loc[df_pix2pix_metrics.shape[0], :] = [*cal_metric(pix2pix_imgs[i], truth_imgs[i])]



    return  df_diff_metrics, df_pix2pix_metrics

df_diff_metrics, df_pix2pix_metrics = get_gloabl_metrics()

100%|██████████| 2712/2712 [08:12<00:00,  5.51it/s]


In [4]:
df_diff_metrics

,ROI_rect_AUC,ROI_rect_MSE,ROI_rect_SSIM,ROI_rect_PSNR,ROI_rect_NRMSE
0,0.985247,206.854294,0.840675,24.974158,0.0
1,0.977673,158.946487,0.798309,26.118294,0.0
2,0.979432,310.395721,0.772483,23.211646,0.0
3,0.978165,250.838379,0.825286,24.136864,0.0
4,0.983061,206.918152,0.811874,24.972818,0.0
...,...,...,...,...,...
2707,0.966604,249.849197,0.718879,24.154024,0.0
2708,0.973429,186.966919,0.770458,25.413156,0.0
2709,0.973338,163.957687,0.796141,25.983486,0.0
2710,0.969113,106.749344,0.824142,27.847151,0.0


In [14]:
df_pix2pix_metrics.mean()

AUC      0.959480
MSE      0.003487
SSIM     0.787219
PSNR    25.229222
dtype: float64

In [15]:
df_diff_metrics.mean()

AUC      0.976254
MSE      0.003133
SSIM     0.820024
PSNR    27.095465
dtype: float64

In [16]:
# 保存csv文件
df_diff_metrics.to_csv("./DiffusionSpinalMRISynthesis/Test_models/metrics/Global_diff_pred_metrics_1231.csv")
df_pix2pix_metrics.to_csv("./DiffusionSpinalMRISynthesis/Test_models/metrics/Global_pix2pix_pred_metrics_1231.csv")

In [17]:
# 保存各项的均值
# 病变区域最小外接矩形的SSIM PSNR AUC MSE NRMSE的均值和方差
# 对于diff和pix2pix
df_diff_sum = df_diff_metrics[['AUC', 'MSE', 'SSIM', 'PSNR']]
df_diff_sum = pd.concat([df_diff_sum.mean(axis=0), df_diff_sum.std(axis=0)], axis=1).rename(columns={0:'diff_mean', 1:'diff_std'})
df_pix2pix_sum = df_pix2pix_metrics[['AUC', 'MSE', 'SSIM', 'PSNR']]
df_pix2pix_sum = pd.concat([df_pix2pix_sum.mean(axis=0), df_pix2pix_sum.std(axis=0)], axis=1).rename(columns={0:'pix2pix_mean', 1:'pix2pix_std'})

df_diff_sum.to_csv("./DiffusionSpinalMRISynthesis/Test_models/metrics/Global_diff_sum_result_1231.csv")
df_pix2pix_sum.to_csv("./DiffusionSpinalMRISynthesis/Test_models/metrics/Global_pix2pix_sum_result_1231.csv")

In [7]:
df_diff_metrics.iloc[:,:].mean(axis=0)

ROI_mean          127.397606
ROI_std            38.502162
backround_mean      2.629492
backround_std       1.594099
ROI_rect_AUC        0.942627
ROI_rect_MSE      625.697222
ROI_rect_SSIM       0.716417
ROI_rect_PSNR      21.197603
ROI_rect_NRMSE      0.244601
ROI_SNR            51.900279
dtype: float64

In [8]:
df_pix2pix_metrics.iloc[:,1:].mean(axis=0)

ROI_mean          111.790998
ROI_std            32.293118
backround_mean      0.661009
backround_std       0.922590
ROI_rect_AUC        0.866164
ROI_rect_MSE      965.099802
ROI_rect_SSIM       0.552505
ROI_rect_PSNR      19.240086
ROI_rect_NRMSE      0.301967
ROI_SNR            65.293787
dtype: float64